In [4]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

/Users/yitonggu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:

# Define imsage Data scale and batch size
shrinking = 4
img_height = int(1944 / shrinking)
img_width = int(2592 / shrinking)
input_shape = (img_height, img_width, 3)
batch_size = 16

# defining dataset directory
dataset_dir = "dataset"

# Get a list of all image file paths
file_paths = []
labels = []

for class_label, class_name in enumerate(os.listdir(dataset_dir)):
    class_dir = os.path.join(dataset_dir, class_name)
    for image_name in os.listdir(class_dir):
        file_paths.append(os.path.join(class_dir, image_name))
        labels.append(class_label)
        # print(image_name, class_label)

# DataFrame with file paths and labels        
data = pd.DataFrame({
    'file_path': file_paths,
    'label': labels
})

# Split the dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# generate datagenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:

train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw'  # Use 'raw' for integer labels
)

test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw'
)


Found 2232 validated image filenames.
Found 558 validated image filenames.


In [7]:
model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # Output layer with appropriate number of units
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


/Users/yitonggu/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
num_epochs = 1

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples,
    epochs= num_epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples,
)


/Users/yitonggu/Library/Python/3.9/lib/python/site-packages/keras/src/backend/tensorflow/nn.py:602: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/Users/yitonggu/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 140/2232 ━━━━━━━━━━━━━━━━━━━━ 32:28 931ms/step - accuracy: 0.3577 - loss: 13.3605

2024-05-07 16:09:36.263112: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


2232/2232 ━━━━━━━━━━━━━━━━━━━━ 144s 62ms/step - accuracy: 0.3592 - loss: 4.8756 - val_accuracy: 0.3746 - val_loss: 1.0969


2024-05-07 16:09:44.552509: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [9]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "pretrained_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)